# Download required modules

In [1]:
!pip install diffusers
!pip install transformers
!pip install accelerate
!pip install dataclasses
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 417.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu1

# Create Config

In [ ]:
from dataclasses import dataclass

class TrainingConfig:
    image_size = 64
    train_batch_size = 16
    eval_batch_size = 16
    num_epochs = 10
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_model_epochs = 5
    mixed_precision = "fp16"
    output_dir = "LDM"

    overwrite_output_dir = True
    seed = 0


config = TrainingConfig()

# Load and Preprocess Data for Training Model

CIFAR10 has images for 10 labels.

Only one label is used for unconditional image synthesis.

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

preprocess = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

def filter_class(dataset, class_label):
    indices = [i for i, target in enumerate(dataset.targets) if target == class_label]
    return Subset(dataset, indices)

class_label = 3

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=preprocess)
filtered_train_dataset = filter_class(train_dataset, class_label)
train_loader = DataLoader(filtered_train_dataset, batch_size=config.train_batch_size, shuffle=True)

Using downloaded and verified file: data/cifar-10-python.tar.gz
Extracting data/cifar-10-python.tar.gz to data


# Set Up for Training

https://huggingface.co/CompVis/ldm-celebahq-256

Set to CUDA

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Load UNet

In [ ]:
from diffusers import UNet2DModel

unet = UNet2DModel.from_pretrained('CompVis/ldm-celebahq-256', subfolder = 'unet')
unet.to(device)

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

unet/config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

An error occurred while trying to fetch CompVis/ldm-celebahq-256: CompVis/ldm-celebahq-256 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

UNet2DModel(
  (conv_in): Conv2d(3, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=224, out_features=896, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=896, out_features=896, bias=True)
  )
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 224, eps=1e-05, affine=True)
          (conv1): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=896, out_features=224, bias=True)
          (norm2): GroupNorm(32, 224, eps=1e-05, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
      (downsamplers): ModuleList(
        (0): Downsample2D(
          (conv): Conv2d(22

Load VQVAE

In [ ]:
from diffusers import VQModel

vqvae = VQModel.from_pretrained('CompVis/ldm-celebahq-256', subfolder = 'vqvae')
vqvae.to(device)

vqvae/config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

An error occurred while trying to fetch CompVis/ldm-celebahq-256: CompVis/ldm-celebahq-256 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/221M [00:00<?, ?B/s]

VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1):

Load Scheduler

In [ ]:
from diffusers import DDIMScheduler

scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")
scheduler.set_timesteps(num_inference_steps=200)

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:245: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddim.DDIMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


scheduler/scheduler_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


Set Optimizer and Learning Rate Scheduler

In [ ]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.Adam(unet.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_loader) * config.num_epochs),
)

Define `train_loop`

`# OWN CODE`



In [ ]:
import torch.nn.functional as F

from accelerate import Accelerator
from tqdm.auto import tqdm
from pathlib import Path
import os


def train_loop(config, unet, vqvae, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize Accelerator
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    unet, vqvae, noise_scheduler, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        unet, vqvae, noise_scheduler, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0
    generator = torch.manual_seed(config.seed)

    # Training
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch+1}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch[0]
            batch_size = clean_images.shape[0]

            # Forward Diffusion
            noise = torch.randn(
                (batch_size, unet.config.in_channels, unet.config.sample_size, unet.config.sample_size),
                generator=generator,
            ).to(device)

            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (batch_size,), device=clean_images.device,
                dtype=torch.int64
            )

            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(unet):
                residual = unet(noisy_images, timesteps)["sample"]
                loss = F.mse_loss(residual, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(unet.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # Save and Model in Specific Epochs
        if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
            fine_tuned_unet = accelerator.unwrap_model(unet)
            fine_tuned_vqvae = accelerator.unwrap_model(vqvae)
            fine_tuned_unet.save_pretrained(os.path.join(config.output_dir, 'unet'))
            fine_tuned_vqvae.save_pretrained(os.path.join(config.output_dir, 'vqvae'))

# Training

In [ ]:
from accelerate import notebook_launcher

args = (config, unet, vqvae, scheduler, optimizer, train_loader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


  0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

...

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

# Save Fine-Tuned Model to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/LDM

mkdir: cannot create directory ‘/content/drive/MyDrive/LDM’: File exists


In [ ]:
!cp -r /content/LDM/. /content/drive/MyDrive/LDM

# Inference for Training Model

Set to CUDA

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

Load Model

In [ ]:
from diffusers import UNet2DModel, DDIMScheduler, VQModel

unet = UNet2DModel.from_pretrained("/content/drive/MyDrive/LDM", subfolder="unet")
vqvae = VQModel.from_pretrained("/content/drive/MyDrive/LDM", subfolder="vqvae")
scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")

unet.to(device)
vqvae.to(device)

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:245: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddim.DDIMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1):

Inference

In [ ]:
!mkdir /content/drive/MyDrive/Output

In [ ]:
import PIL.Image
import numpy as np
from tqdm.auto import tqdm

scheduler.set_timesteps(num_inference_steps=200)

out_dir = '/content/drive/MyDrive/Output'

for i in range(16) :
  # Generate Gaussian Noise
  seed = i

  generator = torch.manual_seed(seed)
  noise = torch.randn(
      (1, unet.config.in_channels, unet.config.sample_size, unet.config.sample_size),
      generator=generator,
  ).to(device)

  image = noise
  for t in tqdm(scheduler.timesteps):
      # Predict Noise Residual
      with torch.no_grad():
          residual = unet(image, t)["sample"]

      # Compute Previous Image
      prev_image = scheduler.step(residual, t, image)["prev_sample"]
      image = prev_image

  # Decode Image
  with torch.no_grad():
      image = vqvae.decode(image)

  image = image.sample

  # Process Image
  image_processed = image.cpu().permute(0, 2, 3, 1)
  image_processed = (image_processed + 1.0) * 127.5
  image_processed = image_processed.clamp(0, 255).numpy().astype(np.uint8)
  image_pil = PIL.Image.fromarray(image_processed[0])

  image_pil.save(os.path.join(out_dir, f"generated_image_{seed:02d}.jpg"))

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

# Evaluation for Training Model

Save Real Image

In [ ]:
!mkdir /content/drive/MyDrive/Real_Img

In [ ]:
import torchvision.transforms as T
from PIL import Image
import cv2
import matplotlib.pyplot as plt

real_img_dir = '/content/drive/MyDrive/Real_Img'

data_loader = DataLoader(filtered_train_dataset, batch_size=config.train_batch_size, shuffle=False)
features, labels = next(iter(data_loader))

for batch_idx, (images, labels) in enumerate(data_loader):
    if batch_idx >= 1:
        break
    for idx, image in enumerate(images):
        filename = os.path.join(real_img_dir, f'real_image_{idx:02d}.jpg')
        image = image.permute(1, 2, 0).numpy()
        image = (image * 255).astype('uint8')
        cv2.imwrite(filename, image)

Save Resized Image

In [ ]:
real_images_dir = "/content/drive/MyDrive/Real_Img"
generated_images_dir= "/content/drive/MyDrive/Output"

In [ ]:
from PIL import Image

def resize_images(image_dir, target_size):
    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(image_dir, filename)
            with Image.open(file_path) as img:
                img = img.resize(target_size, Image.ANTIALIAS)
                img.save(os.path.join(image_dir, 'resize', filename))

In [ ]:
!mkdir /content/drive/MyDrive/Real_Img/resize

In [ ]:
!mkdir /content/drive/MyDrive/Output/resize

In [ ]:
target_size = (256, 256)

resize_images(real_images_dir, target_size)
resize_images(generated_images_dir, target_size)

<ipython-input-23-d832e495a673>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


Evaluate

In [ ]:
real_images_dir = "/content/drive/MyDrive/Real_Img/resize"
generated_images_dir= "/content/drive/MyDrive/Output/resize"

In [ ]:
!pip install torch-fidelity

In [ ]:
import os
import numpy as np
import torch
from scipy.linalg import sqrtm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import inception_v3, Inception_V3_Weights
from PIL import Image

In [ ]:
# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Load Images
def load_images(image_dir):
    images = []
    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(image_dir, filename)
            with Image.open(file_path) as img:
                img = preprocess(img)
                images.append(img)
    return torch.stack(images)

real_images = load_images(real_images_dir)
generated_images = load_images(generated_images_dir)

In [ ]:
# Load InceptionV3 Model
model = inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Identity()
model.eval()

# Set to CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth






  0%|          | 0.00/104M [00:00<?, ?B/s]





  1%|          | 640k/104M [00:00<00:16, 6.39MB/s]





  3%|▎         | 2.75M/104M [00:00<00:06, 15.5MB/s]





  7%|▋         | 7.00M/104M [00:00<00:03, 28.6MB/s]





 15%|█▍        | 15.1M/104M [00:00<00:01, 50.6MB/s]





 28%|██▊       | 28.6M/104M [00:00<00:00, 83.1MB/s]





 45%|████▍     | 46.5M/104M [00:00<00:00, 118MB/s] 





 62%|██████▏   | 64.4M/104M [00:00<00:00, 140MB/s]





 75%|███████▍  | 77.8M/104M [00:00<00:00, 136MB/s]





 87%|████████▋ | 90.8M/104M [00:00<00:00, 123MB/s]





100%|██████████| 104M/104M [00:01<00:00, 97.9MB/s]


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
# Get Activations
def get_activations(images, model, batch_size=8):
    dataloader = DataLoader(images, batch_size=batch_size, shuffle=False)
    activations = []
    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            batch_activations = model(batch).cpu().numpy()
            activations.append(batch_activations)
    return np.concatenate(activations)

real_activations = get_activations(real_images, model)
generated_activations = get_activations(generated_images, model)

In [ ]:
# Calculate mean and covariance
mu_real = np.mean(real_activations, axis=0)
sigma_real = np.cov(real_activations, rowvar=False)
mu_gen = np.mean(generated_activations, axis=0)
sigma_gen = np.cov(generated_activations, rowvar=False)

In [ ]:
# Calculate FID
def calculate_fid(mu1, sigma1, mu2, sigma2, eps=1e-6):
    diff = mu1 - mu2
    diff_squared_sum = np.sum(diff**2)

    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff_squared_sum + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

In [ ]:
# Ensure the Covariance Matrices Are Positive Definite
eps = 1e-6
sigma_real += np.eye(sigma_real.shape[0]) * eps
sigma_gen += np.eye(sigma_gen.shape[0]) * eps

fid_score = calculate_fid(mu_real, sigma_real, mu_gen, sigma_gen)
print(f'FID score: {fid_score}')

FID score: 366.03710543227055


# Create Config

In [ ]:
from dataclasses import dataclass

class TrainingConfig:
    image_size = 64
    train_batch_size = 16
    eval_batch_size = 16
    num_epochs = 10
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_model_epochs = 5
    mixed_precision = "fp16"
    output_dir = "LDM_Test"

    overwrite_output_dir = True
    seed = 0


config = TrainingConfig()

# Load and Preprocess Data for Test Model

CIFAR10 has images for 10 labels.

Only one label is used for unconditional image synthesis.

In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

preprocess = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

def filter_class(dataset, class_label):
    indices = [i for i, target in enumerate(dataset.targets) if target == class_label]
    return Subset(dataset, indices)

class_label = 3

test_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=preprocess)
filtered_test_dataset = filter_class(test_dataset, class_label)
test_loader = DataLoader(filtered_test_dataset, batch_size=config.train_batch_size, shuffle=True)

100%|██████████| 170498071/170498071 [00:12<00:00, 13316104.79it/s]


Extracting data/cifar-10-python.tar.gz to data


# Set Up for Test

Set to CUDA

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Load Fine-Tuned Model

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from diffusers import UNet2DModel, DDIMScheduler, VQModel

unet = UNet2DModel.from_pretrained("/content/drive/MyDrive/LDM", subfolder="unet")
vqvae = VQModel.from_pretrained("/content/drive/MyDrive/LDM", subfolder="vqvae")
scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")

unet.to(device)
vqvae.to(device)

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:245: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddim.DDIMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)

scheduler/scheduler_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1):

Set Optimizer and Learning Rate Scheduler

In [8]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.Adam(unet.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(test_loader) * config.num_epochs),
)

Define `train_loop`

`# OWN CODE`

In [9]:
import torch.nn.functional as F

from accelerate import Accelerator
from tqdm.auto import tqdm
from pathlib import Path
import os


def train_loop(config, unet, vqvae, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize Accelerator
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    unet, vqvae, noise_scheduler, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        unet, vqvae, noise_scheduler, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0
    generator = torch.manual_seed(config.seed)

    # Training
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch+1}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch[0]
            batch_size = clean_images.shape[0]

            # Forward Diffusion
            noise = torch.randn(
                (batch_size, unet.config.in_channels, unet.config.sample_size, unet.config.sample_size),
                generator=generator,
            ).to(device)

            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (batch_size,), device=clean_images.device,
                dtype=torch.int64
            )

            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(unet):
                residual = unet(noisy_images, timesteps)["sample"]
                loss = F.mse_loss(residual, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(unet.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # Save and Model in Specific Epochs
        if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
            fine_tuned_unet = accelerator.unwrap_model(unet)
            fine_tuned_vqvae = accelerator.unwrap_model(vqvae)
            fine_tuned_unet.save_pretrained(os.path.join(config.output_dir, 'unet'))
            fine_tuned_vqvae.save_pretrained(os.path.join(config.output_dir, 'vqvae'))

# Training for Test

In [10]:
from accelerate import notebook_launcher

args = (config, unet, vqvae, scheduler, optimizer, test_loader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


  0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

...

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

# Save Test Model to Google Drive

In [11]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!mkdir /content/drive/MyDrive/LDM_Test

In [16]:
!cp -r /content/LDM_Test/. /content/drive/MyDrive/LDM_Test

# Inference for Test Model

Set to CUDA

In [14]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

Load Model

In [17]:
from diffusers import UNet2DModel, DDIMScheduler, VQModel

unet = UNet2DModel.from_pretrained("/content/drive/MyDrive/LDM_Test", subfolder="unet")
vqvae = VQModel.from_pretrained("/content/drive/MyDrive/LDM_Test", subfolder="vqvae")
scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")

unet.to(device)
vqvae.to(device)

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:245: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddim.DDIMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1):

Inference

In [18]:
!mkdir /content/drive/MyDrive/Output_Test

In [19]:
import PIL.Image
import numpy as np
from tqdm.auto import tqdm

scheduler.set_timesteps(num_inference_steps=200)

out_dir = '/content/drive/MyDrive/Output_Test'

for i in range(16) :
  # Generate Gaussian Noise
  seed = i

  generator = torch.manual_seed(seed)
  noise = torch.randn(
      (1, unet.config.in_channels, unet.config.sample_size, unet.config.sample_size),
      generator=generator,
  ).to(device)

  image = noise
  for t in tqdm(scheduler.timesteps):
      # Predict Noise Residual
      with torch.no_grad():
          residual = unet(image, t)["sample"]

      # Compute Previous Image
      prev_image = scheduler.step(residual, t, image)["prev_sample"]
      image = prev_image

  # Decode Image
  with torch.no_grad():
      image = vqvae.decode(image)

  image = image.sample

  # Process Image
  image_processed = image.cpu().permute(0, 2, 3, 1)
  image_processed = (image_processed + 1.0) * 127.5
  image_processed = image_processed.clamp(0, 255).numpy().astype(np.uint8)
  image_pil = PIL.Image.fromarray(image_processed[0])

  image_pil.save(os.path.join(out_dir, f"generated_image_{seed:02d}.jpg"))

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

# Evaluation for Test Model

Save Real Image

In [20]:
!mkdir /content/drive/MyDrive/Real_Img_Test

In [21]:
import torchvision.transforms as T
from PIL import Image
import cv2
import matplotlib.pyplot as plt

real_img_dir = '/content/drive/MyDrive/Real_Img_Test'

data_loader = DataLoader(filtered_test_dataset, batch_size=config.train_batch_size, shuffle=False)
features, labels = next(iter(data_loader))

for batch_idx, (images, labels) in enumerate(data_loader):
    if batch_idx >= 1:
        break
    for idx, image in enumerate(images):
        filename = os.path.join(real_img_dir, f'real_image_{idx:02d}.jpg')
        image = image.permute(1, 2, 0).numpy()
        image = (image * 255).astype('uint8')
        cv2.imwrite(filename, image)

Save Resized Image

In [22]:
real_images_dir = "/content/drive/MyDrive/Real_Img_Test"
generated_images_dir= "/content/drive/MyDrive/Output_Test"

In [23]:
from PIL import Image

def resize_images(image_dir, target_size):
    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(image_dir, filename)
            with Image.open(file_path) as img:
                img = img.resize(target_size, Image.ANTIALIAS)
                img.save(os.path.join(image_dir, 'resize', filename))

In [24]:
!mkdir /content/drive/MyDrive/Real_Img_Test/resize

In [25]:
!mkdir /content/drive/MyDrive/Output_Test/resize

In [26]:
target_size = (256, 256)

resize_images(real_images_dir, target_size)
resize_images(generated_images_dir, target_size)

<ipython-input-23-d832e495a673>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


Evaluate

In [27]:
real_images_dir = "/content/drive/MyDrive/Real_Img_Test/resize"
generated_images_dir= "/content/drive/MyDrive/Output_Test/resize"

In [28]:
import os
import numpy as np
import torch
from scipy.linalg import sqrtm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import inception_v3, Inception_V3_Weights
from PIL import Image

In [29]:
# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [30]:
# Load Images
def load_images(image_dir):
    images = []
    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(image_dir, filename)
            with Image.open(file_path) as img:
                img = preprocess(img)
                images.append(img)
    return torch.stack(images)

real_images = load_images(real_images_dir)
generated_images = load_images(generated_images_dir)

In [31]:
# Load InceptionV3 Model
model = inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Identity()
model.eval()

# Set to CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth










  0%|          | 0.00/104M [00:00<?, ?B/s]









 13%|█▎        | 13.6M/104M [00:00<00:00, 143MB/s]









 30%|██▉       | 31.1M/104M [00:00<00:00, 166MB/s]









 47%|████▋     | 48.4M/104M [00:00<00:00, 173MB/s]









 64%|██████▍   | 66.4M/104M [00:00<00:00, 179MB/s]









 82%|████████▏ | 84.8M/104M [00:00<00:00, 184MB/s]









100%|██████████| 104M/104M [00:00<00:00, 180MB/s]


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [32]:
# Get Activations
def get_activations(images, model, batch_size=8):
    dataloader = DataLoader(images, batch_size=batch_size, shuffle=False)
    activations = []
    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            batch_activations = model(batch).cpu().numpy()
            activations.append(batch_activations)
    return np.concatenate(activations)

real_activations = get_activations(real_images, model)
generated_activations = get_activations(generated_images, model)

In [33]:
# Calculate mean and covariance
mu_real = np.mean(real_activations, axis=0)
sigma_real = np.cov(real_activations, rowvar=False)
mu_gen = np.mean(generated_activations, axis=0)
sigma_gen = np.cov(generated_activations, rowvar=False)

In [34]:
# Calculate FID
def calculate_fid(mu1, sigma1, mu2, sigma2, eps=1e-6):
    diff = mu1 - mu2
    diff_squared_sum = np.sum(diff**2)

    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff_squared_sum + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

In [35]:
# Ensure the Covariance Matrices Are Positive Definite
eps = 1e-6
sigma_real += np.eye(sigma_real.shape[0]) * eps
sigma_gen += np.eye(sigma_gen.shape[0]) * eps

fid_score = calculate_fid(mu_real, sigma_real, mu_gen, sigma_gen)
print(f'FID score: {fid_score}')

FID score: 327.76980818052584
